In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA

from scipy.integrate import solve_ivp
from scipy import integrate
from scipy.linalg import null_space

from scipy.optimize import fsolve
from scipy.integrate import solve_bvp

import pickle
import pandas as pd
import cmath

from itertools import chain
import csv

In [ ]:
from concurrent.futures import ProcessPoolExecutor

In [ ]:
%matplotlib notebook

In [ ]:
# length
L = 0.6

# orbital number
l = 1

# coupling srength left/right
t_L = 0.41 # 0.99
t_R = t_L

# effective mass
mx = 30 # 30
Delta = 1

In [ ]:
phi_0 = np.linspace(0.003, 2*np.pi, num=201)
phi_0[0] = 0.001
phi_0[-1] -= 0.001
phi_0[100] += 0.001

mid = len(phi_0)//2

# # if needed choose a spesific region of phases too observe the Mpemba effect closer in this region

# range1 = np.linspace(0, 0.3 * np.pi, num=50)  # Exclude endpoint if desired
# range2 = np.linspace(0.7 * np.pi, 1 * np.pi, num=50)         # Include endpoint if desired

# # Combine the two ranges
# phi_0 = np.concatenate((range1, range2))

In [ ]:
lambda_coupl = 0.1 # Andreev-continuum coupling rate
damping_param = 0.1 # 0.01
Omega = 0.01 #0.2

T_ferm = 0.5
beta_ferm = 1/T_ferm
T_bos = 1/5

alpha_0 = 0.001 # 0.0001
omega_c = 1

In [ ]:
def bose_distrib(x, T):
    """ Bose distribution function that handles large x/T to avoid overflow in the exponent """  

    y = x / T
    cutoff = 500.0  # cutoff to avoid overflow in exp
    
    if y >= 0:
        if y > cutoff:
            return 0.0  # for large y, n_B -> 0
        else:
            return 1.0 / np.expm1(y)  # expm1(y) = exp(y) - 1
    else:
        y = -y
        if y > cutoff:
            return -1.0  # for large negative y, expression goes to -1 (since n_B -> 0)
        else:
            return -1.0 - 1.0 / np.expm1(y)

In [ ]:
def fermi_distrib(x, T):
    """ Fermi distribution function that handles large x/T to avoid overflow in the exponent """    
    
    y = x / T
    cutoff = 500.0  # cutoff to avoid overflow in exp
    
    if y > cutoff:
        return 0.0  # for large positive y, 1/(exp(y)+1) -> 0
    elif y < -cutoff:
        return 1.0  # for large negative y, 1/(exp(y)+1) -> 1
    else:
        return 1.0 / (np.exp(y) + 1.0)

In [ ]:
# spectral density function

def spectral_density(x, lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    J = lambda_coupl**2 * damping_param * (1/((x - Omega)**2 + damping_param**2/4) - \
                                                   1/((x + Omega)**2 + damping_param**2/4))
    
    J_ohm = alpha_0 * x * np.exp(-np.abs(x)/omega_c)
    
    return J + J_ohm

### Finding free dot eigenenergies and eigenfunctions -  hopping parameters

In [ ]:
def fun(x, y, p):
    k = p[0]
    return np.vstack((y[1], -k**2 * y[0]))

In [ ]:
# boundary condition

def bc(ya, yb, p):
#     k = p[0]

# last element - normalization condition (knowing that I will get cos function as a solution)
    return np.array([ya[1], yb[1], yb[0] - np.sqrt(2/L)])

In [ ]:
# Set up the initial mesh and guess for y

x = np.linspace(-L/2, L/2, 10)
y = np.zeros((2, x.size))
y[0, 1] = 1
y[0, 6] = -1

In [ ]:
sol_test = solve_bvp(fun, bc, x, y, p=[np.pi/L])

In [ ]:
# function to find tn at the contacts for the specified l 

def find_tn(l):
    tn = np.zeros((l, 2))
    
    x = np.linspace(-L/2, L/2, 10)
    y = np.zeros((2, x.size))
    y[0, 1] = 1
    y[0, 6] = -1
    
    for i in range(l):
        sol_i = solve_bvp(fun, bc, x, y, p=[(i+1)*np.pi/L])
        
        # tn[l, -L/2]
        tn[i, 0] = sol_i.sol(-L/2)[0]
        
        # tn[l, L/2]
        tn[i, 1] = sol_i.sol(L/2)[0]
        
    return tn

In [ ]:
x_plot = np.linspace(-L/2, L/2, 100)
y_plot = sol_test.sol(x_plot)[0]
plt.plot(x_plot, y_plot)
plt.xlabel("x")
plt.ylabel("y")

# compare the obtained solution with the analytical solution given by:
# sqrt(2/L) * cos(p_n(x-L/2)), p_n = pi*n/L

plt.scatter(x_plot, np.sqrt(2/L)*np.cos(np.pi/L*(x_plot - L/2)), s=10)
plt.grid()

In [ ]:
# hopping parameters

tn = find_tn(l)
tn

In [ ]:
# Calculate G_L, G_R matrices from tn matrix (2 columns of tn are the hopping parameters (L/R) of each level)

col_L = tn[:, 0]

Gamma_L = t_L**2 * np.outer(col_L, col_L)  # Compute the outer product to form a square matrix
 
col_R = tn[:, 1]
Gamma_R = t_R**2 * np.outer(col_R, col_R) 

In [ ]:
# free dot energies

l_values = np.arange(2, l+2)  
eps_n = (l_values * np.pi / L)**2 / (2 * mx)

In [ ]:
# transparency

transp = 4*Gamma_R[0,0]**2/(4*Gamma_R[0,0]**2+eps_n[0]**2)
transp

### Finding Andreev states

In [ ]:
def G_inv(x, i):
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda = -1 / np.sqrt(Delta**2 - x**2) \
            * (x * np.block([[(Gamma_L + Gamma_R), zero_m], [zero_m, (Gamma_L + Gamma_R)]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * Gamma_L], \
                                [np.exp(1j * s_L * phi_0[i]/2) * Gamma_L, zero_m]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * Gamma_R], \
                                [np.exp(1j * s_R * phi_0[i]/2) * Gamma_R, zero_m]]))
    
    G_inv = x * np.eye(2) - np.block([[np.diag(eps_n), zero_m], [zero_m, -np.diag(eps_n)]]) - Lambda
    
    return LA.det(G_inv)   

In [ ]:
roots = np.zeros((len(phi_0), 2*l)) # note: the energies are real-valued!

for i in range(len(phi_0)):
  
    roots[i, 0] = fsolve(G_inv, 0.99, args=(i))  
    roots[i, 1] = fsolve(G_inv, -0.99, args=(i))   

In [ ]:
# Save the obtained ABS energies as a txt file

data = pd.DataFrame({
    "phi_0": phi_0/np.pi,
    "ABS_energy": roots[:,0]
})

# Save the DataFrame to a text file (tab-separated)
data.to_csv("ABS_0.1.txt", sep="\t", index=False)

In [ ]:
plt.plot(phi_0/np.pi, roots[:,0], linewidth=1.5)
plt.plot(phi_0/np.pi, roots[:,1], linewidth=1.5)


plt.grid()
plt.xlabel(r'$\varphi_0$, $\pi$', fontsize=12)
plt.ylabel(r'$E_1$, $\Delta$', fontsize=12)
plt.title(r"$L={}\Delta^{{-1}}, t_L={}, t_R={}, m_{{x}} = {}\Delta,$"\
          .format(L, t_L, t_R, mx), fontsize=13)


### Finding transition rates ABS -> ABS

In [ ]:
# diagonalizing the Hamiltonian: get eigenenergies - ABS, and eigenfunctions

def diagonal(x):
    
    eigval_arr = np.zeros((len(phi_0), 2*l))
    eigvec_arr = np.zeros((len(phi_0), 2*l, 2*l), dtype='complex')
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    for i in range(len(phi_0)):
    
        Lambda = -1 / np.sqrt(Delta**2 - x[i]**2) \
            * (x[i] * np.block([[(Gamma_L + Gamma_R), zero_m], [zero_m, (Gamma_L + Gamma_R)]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * Gamma_L], \
                                [np.exp(1j * s_L * phi_0[i]/2) * Gamma_L, zero_m]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * Gamma_R], \
                                [np.exp(1j * s_R * phi_0[i]/2) * Gamma_R, zero_m]]))
    
        H = np.block([[np.diag(eps_n), zero_m], [zero_m, -np.diag(eps_n)]]) + Lambda
    
        
        eigval_arr[i], eigvec_arr[i] = LA.eigh(H)
    
    return eigval_arr, eigvec_arr 

In [ ]:
eigv_0, eigvec_0 = diagonal(roots[:, 0])
eigv_1, eigvec_1 = diagonal(roots[:, 1])

In [ ]:
# has to be the same plot as the ABS energies obtained from fsolve!

plt.plot(phi_0[:]/np.pi, eigv_0[:, 1])
plt.plot(phi_0[:]/np.pi, eigv_1[:, 0])

plt.grid()
plt.xlabel(r'$\varphi_0$, $\pi$', fontsize=12)
plt.ylabel(r'$E_1$, $\Delta$', fontsize=12)
plt.title(r"$L={}\Delta^{{-1}}, t_L={}, t_R={}, m_{{x}} = {}\Delta,$"\
          .format(L, t_L, t_R, mx), fontsize=13)

In [ ]:
# Eigenvectors corresponding to the Andreev energies

eta_0 = eigvec_0[:, :, 1]
eta_1 = eigvec_1[:, :, 0]


eta_arr = np.stack([eta_0, eta_1], axis=0)

In [ ]:
def curr_matr(x, i):
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda_L = x * np.block([[Gamma_L , zero_m], [zero_m, Gamma_L]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * Gamma_L], \
                                [np.exp(1j * s_L * phi_0[i]/2) * Gamma_L, zero_m]])
    
    Lambda_R = x * np.block([[Gamma_R , zero_m], [zero_m, Gamma_R]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * Gamma_R], \
                                [np.exp(1j * s_R * phi_0[i]/2) * Gamma_R, zero_m]])
    
    I = -1 / np.sqrt(Delta**2 - x**2) / (2*1j) * (s_L * Lambda_L + s_R * Lambda_R)
    
    return I

In [ ]:
def curr_nl(E_n, E_lamb, eta_n, eta_lamb, i):
    
    tau_z = np.array([[1, 0], [0, -1]])
    
    matr = tau_z @ curr_matr(E_lamb, i) - curr_matr(E_n, i) @ tau_z
    
    return np.conj(eta_n) @ matr @ eta_lamb    

In [ ]:
I_nl = np.zeros((len(phi_0), 2*l, 2*l), dtype='complex')
G_nl = np.zeros((len(phi_0), 2*l, 2*l))

for i in range(len(phi_0)):
    for n in range(2*l):
        for lamd in range(2*l):
            I_nl[i, n ,lamd] = curr_nl(roots[i, n], roots[i, lamd], eta_arr[n, i], eta_arr[lamd, i], i)

In [ ]:
# Plot the current matrix elements

plt.plot(phi_0/np.pi, np.abs(I_nl[:, 0, 0]))
plt.plot(phi_0/np.pi, np.abs(I_nl[:, 1, 0]))

In [ ]:
for i in range(len(phi_0)):
    for n in range(2*l):
        for lamd in range(2*l):
            
            energy_diff = roots[i, n] - roots[i, lamd]
            J_spect = spectral_density(energy_diff, lambda_coupl, alpha_0, damping_param, Omega, omega_c)
            bose = bose_distrib(energy_diff, T_bos)
            
            G_nl[i, lamd, n] = 2*np.pi * np.abs(I_nl[i, n, lamd])**2 * J_spect * bose 
            
            if math.isnan(G_nl[i, lamd, n]):
                G_nl[i, lamd, n] = 0

In [ ]:
G_AL_pp = np.zeros((len(phi_0), 2, 2))

G_AL_pm = np.zeros((len(phi_0), 2, 2))
G_AL_pm[:, 0, 1] = G_nl[:, 0, 1]
G_AL_pm[:, 1, 0] = G_nl[:, 0, 1]


G_AL_mp = np.zeros((len(phi_0), 2, 2))
G_AL_mp[:, 0, 1] = G_nl[:, 1, 0]
G_AL_mp[:, 1, 0] = G_nl[:, 1, 0]

In [ ]:
plt.plot(phi_0/np.pi, G_AL_pm[:, 0, 1], label=r'$\Gamma_{1\bar1}$')
plt.plot(phi_0/np.pi, G_AL_mp[:, 1, 0], label=r'$\Gamma_{\bar1 1}$')


plt.legend()
plt.grid()

### Finding transition rates ABS -> continuum: DOS

In [ ]:
# # DOS matrix

def spectr(x, i):
    
    eigval_arr = np.zeros((2))
    eigvec_arr = np.zeros((2, 2), dtype='complex')
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda = 1 / np.sqrt(x**2 - Delta**2) \
        * (x * np.block([[(Gamma_L + Gamma_R), zero_m], [zero_m, (Gamma_L + Gamma_R)]]) \
        - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * Gamma_L], \
                            [np.exp(1j * s_L * phi_0[i]/2) * Gamma_L, zero_m]]) \
        - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * Gamma_R], \
                            [np.exp(1j * s_R * phi_0[i]/2) * Gamma_R, zero_m]]))

    G_R = LA.inv(x * np.eye(2) - np.block([[np.diag(eps_n), zero_m], [zero_m, -np.diag(eps_n)]]) \
          + 1j * np.sign(x) * Lambda)

    G_A = np.transpose(np.conj(G_R))

    Spectr = 1j * (G_R - G_A)

    eigval_arr, eigvec_arr = LA.eigh(Spectr)
    
    return eigval_arr, eigvec_arr    

In [ ]:
eigv_c = np.zeros((len(phi_0), 2))
eigvect_c = np.zeros((len(phi_0), 2, 2), dtype='complex')

for i in range(len(phi_0)):
    eigv_c[i], eigvect_c[i] = spectr(1.5, i)

In [ ]:
plt.plot(phi_0/np.pi, eigv_c[:, 0])
plt.plot(phi_0/np.pi, eigv_c[:, 1])

In [ ]:
# current matrix for continuum states E > Delta - retarded square root in the denominator

def curr_matr_cont(x, i):
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda_L = x * np.block([[Gamma_L , zero_m], [zero_m, Gamma_L]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * Gamma_L], \
                                [np.exp(1j * s_L * phi_0[i]/2) * Gamma_L, zero_m]])
    
    Lambda_R = x * np.block([[Gamma_R , zero_m], [zero_m, Gamma_R]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * Gamma_R], \
                                [np.exp(1j * s_R * phi_0[i]/2) * Gamma_R, zero_m]])
    
    I = -1 / (-1j * np.sign(x) * np.sqrt(x**2 - Delta**2)) / (2*1j) \
        * (s_L * Lambda_L + s_R * Lambda_R)
    
    return I

In [ ]:
# continuum current matrix element

def curr_cont_nl(E, E_lamd, psi_n, eta_lamd, i):
    
    tau_z = np.array([[1, 0], [0, -1]])
    
    matr = tau_z @ curr_matr(E_lamd, i) - curr_matr_cont(E, i) @ tau_z
    
    return np.conj(psi_n) @ matr @ eta_lamd    

In [ ]:
def integr_gamma_out(x, E_lamd, eta_lamd, i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    E = x + E_lamd
    
    # 4 eigenvalues, eigenvectors for continuum state
    eigv_E, eigvect_E = spectr(E, i)
    
    # current elements
    I_nl = np.zeros((2), dtype='complex')
    
    for n in range(2): 
        I_nl = curr_cont_nl(E, E_lamd, eigvect_E[:, n], eta_lamd, i)
        
    sum_n = np.sum(eigv_E * np.abs(I_nl)**2)
    
    n_F = fermi_distrib(E, T_ferm)
    n_B = bose_distrib(x, T_bos)
    J_sp = spectral_density(x, lambda_coupl, alpha_0, damping_param, Omega, omega_c)
    
    return J_sp * n_B * (1 - n_F) * sum_n 
    

In [ ]:
def integr_gamma_in(x, E_lamd, eta_lamd, i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    E = x + E_lamd
    
    # 4 eigenvalues, eigenvectors for continuum state
    eigv_E, eigvect_E = spectr(E, i)
    
    # current elements
    I_nl = np.zeros((2), dtype='complex')
    
    for n in range(2): 
        I_nl = curr_cont_nl(E, E_lamd, eigvect_E[:, n], eta_lamd, i)
        
    sum_n = np.sum(eigv_E * np.abs(I_nl)**2)
    
    n_F = fermi_distrib(E, T_ferm)
    n_B = bose_distrib(x, T_bos)
    J_sp = spectral_density(x, lambda_coupl, alpha_0, damping_param, Omega, omega_c)
    
    return J_sp * (n_B + 1) * n_F * sum_n 
    

In [ ]:
def compute_integrals(i, lamd, Delta, roots, eta_arr, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    Gamma_out_pc = integrate.quad(integr_gamma_out, Delta - roots[i, lamd], np.infty,
                                  args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    Gamma_out_mc = integrate.quad(integr_gamma_out, -np.infty, -Delta - roots[i, lamd],
                                  args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    Gamma_in_pc = integrate.quad(integr_gamma_in, Delta - roots[i, lamd], np.infty,
                                 args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    Gamma_in_mc = integrate.quad(integr_gamma_in, -np.infty, -Delta - roots[i, lamd],
                                 args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    
    # Return results as a tuple
    return (i, lamd, Gamma_out_pc, Gamma_out_mc, Gamma_in_pc, Gamma_in_mc)

In [ ]:
Gamma_out_pc = np.zeros((len(phi_0), 1))
Gamma_out_mc = np.zeros_like(Gamma_out_pc) 
Gamma_in_pc = np.zeros_like(Gamma_out_pc)
Gamma_in_mc = np.zeros_like(Gamma_out_pc)

# Use ProcessPoolExecutor for parallel execution
with ProcessPoolExecutor() as executor:
    
    # Create a list of futures by submitting tasks for parallel execution
    futures = [executor.submit(compute_integrals, i, lamd, Delta, roots, eta_arr, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c)
               for i in range(mid+1) for lamd in reversed(range(1))]
    
    # Collect results as they are completed
    for future in futures:
        i, lamd, out_pc, out_mc, in_pc, in_mc = future.result()
        
        # Store results in the appropriate arrays
        Gamma_out_pc[i, lamd] = out_pc
        Gamma_out_mc[i, lamd] = out_mc
        Gamma_in_pc[i, lamd] = in_pc
        Gamma_in_mc[i, lamd] = in_mc

In [ ]:
Gamma_in = (Gamma_in_pc + Gamma_in_mc) * 2*np.pi
Gamma_out = (Gamma_out_pc + Gamma_out_mc) * 2*np.pi

Gamma_in[len(phi_0)//2+1:] = Gamma_in[:len(phi_0)//2][::-1]
Gamma_out[len(phi_0)//2+1:] = Gamma_out[:len(phi_0)//2][::-1]

Gamma_in = np.tile(Gamma_in, 2)
Gamma_out = np.tile(Gamma_out, 2)

In [ ]:
plt.plot(phi_0/np.pi, Gamma_out[:, 0], label=r'$\Gamma^{out}_1$')
plt.plot(phi_0/np.pi, Gamma_in[:, 0], label=r'$\Gamma^{in}_1$')

plt.plot(phi_0/np.pi, Gamma_out[:, 1], label=r'$\Gamma^{out}_{2}$')
plt.plot(phi_0/np.pi, Gamma_in[:, 1], label=r'$\Gamma^{in}_{2}$')

plt.legend()

## Lindblad euqation: steady state

In [ ]:
N = 2
N_state = 2**N

In [ ]:
# function that returns the momentum state (as string) for printing

def print_state(n):
    
    if len(bin(n)[:1:-1]) < N:
        new_bin = bin(n)[:1:-1]
        for i in range(N - len(bin(n)[:1:-1])):
            new_bin += '0'
            
        return new_bin
    else:
        return bin(n)[:1:-1]

In [ ]:
# the momentum vectors < q_i0 ... q_iNk | rho | q_j0 ... q_jNk >, where q's can be either 0 or 1 
# in case of fermions - by writing each i and j of the density matrix in the binary representation
# we get all possible combinations of occupied/non-occupied Nk momentum states

# this function finds whether the q'_iq entry (q = [0, Nk-1] index of the entry in the vector) is occupied or not 
def n(q, i):
    if q > len(bin(i)[:1:-1])-1:
        return 0
    else:
        return int(bin(i)[:1:-1][q])

In [ ]:
lvl = np.array(range(N))
states = np.array(range(N_state))

n_v = np.vectorize(n)

nf_h = np.array([n_v(lvl, si) for si in states])

In [ ]:
# steady state
Steady_M = np.zeros((len(phi_0), N_state, N_state), dtype='float64') 

for j in range(len(phi_0)):
    
    # loop over the raws - as many as there are states - 16
    for i in range(N_state):

        # loop over 4 levels
        for mu in range(2):

            n_mu = nf_h[i, mu]

            # G_in
            Steady_M[j, i, i + (-1)**n_mu * 2**mu] += n_mu * (Gamma_in[j, mu] )

            # G_out
            Steady_M[j, i, i + (-1)**n_mu * 2**mu] += (1-n_mu) * (Gamma_out[j, mu] )

            # G_in, G_out
            Steady_M[j, i, i] -= n_mu * (Gamma_out[j, mu]) + \
                                (1-n_mu) * (Gamma_in[j, mu] )

    #             loop over lambda' != lamda
            for nu in chain(range(mu), range(mu+1, 2)):

                n_nu = nf_h[i, nu]

                Steady_M[j, i, i + (-1)**n_mu * 2**mu + (-1)**n_nu * 2**nu] += \
                                n_mu * (1-n_nu) * G_AL_pp[j, nu, mu] + \
                                n_mu * n_nu * G_AL_mp[j, nu, mu] + \
                                (1-n_mu) * (1-n_nu) * G_AL_pm[j, nu, mu]

                Steady_M[j, i, i] -= (1-n_mu) * n_nu * G_AL_pp[j, nu, mu] + \
                                         (1-n_mu) * (1-n_nu) * G_AL_mp[j, nu, mu] +\
                                         n_mu * n_nu * G_AL_pm[j, nu, mu]


In [ ]:
ns = np.zeros((len(phi_0), 4), dtype='float64')

for i in range(len(phi_0)):
    ns[i] = null_space(Steady_M[i], rcond=10e-16)[:,0]
    
    
steady_occup = ns
steady_occup /= np.sum(steady_occup, axis=1, keepdims=True)

In [ ]:
for i in range(N_state):
    print(nf_h[i], '\t', steady_occup[1, i])

In [ ]:
plt.plot(phi_0[:mid]/np.pi, steady_occup[:mid, 0], label=r'$P_{|0\rangle}$')
plt.plot(phi_0[:mid]/np.pi, steady_occup[:mid, 1], label=r'$P_{|\uparrow \downarrow \rangle}$')
plt.plot(phi_0[:mid]/np.pi, steady_occup[:mid, 3], label=r'$P_{|1\rangle}$')



plt.xlabel(r'$\varphi_0, \pi$', fontsize=12)
plt.grid()
plt.legend()
plt.title(r"$T_{{qp}}^{{-1}}={}, T_b^{{-1}}={}, \Omega={}, \eta_{{d}} = {},$"\
          .format(beta_ferm, 1/T_bos, Omega, damping_param), fontsize=13)
plt.ylim(-0.05, 1)

In [ ]:
plt.plot(roots[:mid, 0], steady_occup[:mid, 0], label=r'$P_{|0\rangle}$')
plt.plot(roots[:mid, 0], steady_occup[:mid, 1], label=r'$P_{|\uparrow \downarrow \rangle}$')
plt.plot(roots[:mid, 0], steady_occup[:mid, 3], label=r'$P_{|1\rangle}$')



plt.xlabel(r'$E_1, \Delta$', fontsize=12)
plt.grid()
plt.legend()
plt.title(r"$T_{{qp}}^{{-1}}={}, T_b^{{-1}}={}, \Omega={}, \eta_{{d}} = {},$"\
          .format(beta_ferm, 1/T_bos, Omega, damping_param), fontsize=13)

plt.ylim(-0.05, 1)

In [ ]:
# Save the occupation prob as a txt file

data = pd.DataFrame({
    "phi_0": phi_0/np.pi,
    "P_00": steady_occup[:, 0],
    "P_01/P_10": steady_occup[:, 1],
    "P_11": steady_occup[:, 3]
})

# Save the DataFrame to a text file (tab-separated)
data.to_csv("Popul_1_0.1.txt", sep="\t", index=False)

In [ ]:
def rhs1(t, P, i_eq):

    # find derivatives of N_state elements
    dP = np.zeros(N_state)
        
    dP = Steady_M[i_eq] @ P
        
    return dP

In [ ]:
def compute_tau(i1, i2):
    
    P0 = steady_occup[i1]
    sol_ = solve_ivp(rhs1, (0.0, 1e13), P0, method='LSODA', dense_output=True, args=(i2,))
    ts_ = sol_.t
    ys_ = np.stack(sol_.y)
    
    tau = 0  # Default value for tau
    for j in range(len(ts_)):
        if np.sum(np.abs(ys_[:, j] - steady_occup[i2, :])) < 1e-3:
            tau = ts_[j]
            break
            
    return i1, i2, tau

In [ ]:
tau_arr = np.zeros((len(phi_0), len(phi_0)))

# Parallel execution
with ProcessPoolExecutor() as executor:
    # Submit tasks for all (i1, i2) pairs
    futures = [executor.submit(compute_tau, i1, i2) for i1 in range(mid+1) for i2 in range(mid+1)]
    
    # Collect results and populate tau_arr
    for future in futures:
        i1, i2, tau = future.result()
        tau_arr[i1, i2] = tau

In [ ]:
N = len(phi_0)
for i in range(mid + 1):
    for j in range(mid + 1):
        value = tau_arr[i, j]
        
        # Reflect to the other quadrants
        tau_arr[i, N - 1 - j] = value          # Top-right
        tau_arr[N - 1 - i, j] = value          # Bottom-left
        tau_arr[N - 1 - i, N - 1 - j] = value  # Bottom-right

In [ ]:
# Plotting the initial trace distance

i_st=1
i_f = mid

P0 = steady_occup[:, np.newaxis]
init_trace_dist = np.sum(np.abs(P0 - steady_occup), axis=2)

plt.imshow(init_trace_dist[i_st+1:i_f, i_st+1:i_f], extent=[phi_0[i_st+1]/np.pi, phi_0[i_f]/np.pi,\
                                                   phi_0[i_st+1]/np.pi, phi_0[i_f]/np.pi],
           origin='lower', aspect='auto', cmap='turbo', vmin=0)
plt.title(r"$L = {}\Delta^{{-1}}, \tau = {}, \eta_d = {}, $"\
          .format(L, np.round(transp, 2), damping_param)+'\n'+
          r"$\Omega={}\Delta, T_{{qp}}^{{-1}} = {}\Delta, T_b^{{-1}} = {}\Delta$"\
          .format(Omega, beta_ferm, 1/T_bos), fontsize=13)

cbar = plt.colorbar()
cbar.set_label(r'$\mathcal{D}_T(0)$')  # Color bar label in log scale

# # Uncomment to add contour lines
# contour_levels = np.linspace(np.min(init_trace_dist[i_st:i_f, i_st:i_f]), 
#                              np.max(init_trace_dist[i_st:i_f, i_st:i_f]), 15)
# contour = plt.contour(init_trace_dist[i_st:i_f, i_st:i_f],
#                       levels=contour_levels,
#                       extent=[phi_0[i_st]/np.pi, phi_0[i_f]/np.pi,
#                               phi_0[i_st]/np.pi, phi_0[i_f]/np.pi],
#                       colors='black', linewidths=0.5)

# # Add contour labels (optional)
# plt.clabel(contour, inline=True, fontsize=8, fmt="%.2f")

# plt.axvline(x=0.65, color='red', linestyle='-', linewidth=1)
# plt.axhline(y=0.99, color='red', linestyle='-', linewidth=1)


plt.xlabel(r'$\varphi_{final}, \pi$')
plt.ylabel(r'$\varphi_{init}, \pi$')

plt.show()

In [ ]:
# Plotting the relaxation time tau

plt.imshow(tau_arr[i_st+25:i_f, i_st+25:i_f], extent=[phi_0[i_st+25]/np.pi, phi_0[i_f]/np.pi,\
                                        phi_0[i_st+25]/np.pi, phi_0[i_f]/np.pi],\
       origin='lower', aspect='auto', cmap='turbo',)


plt.colorbar(label=r'$\tau, \Delta$')
plt.xlabel(r'$\varphi_{final}, \pi$')
plt.ylabel(r'$\varphi_{init}, \pi$')

plt.title(r"$L = {}\Delta^{{-1}}, \tau = {}, \eta_d = {}, $"\
          .format(L, np.round(transp, 2), damping_param)+'\n'+
          r"$\Omega={}\Delta, T_{{qp}}^{{-1}} = {}\Delta, T_b^{{-1}} = {}\Delta$"\
          .format(Omega, beta_ferm, 1/T_bos), fontsize=13)

plt.show()

In [ ]:
# Plotting the logsrithm of the relaxation time tau

tau_arr_log = np.log10(np.maximum(tau_arr, 10))

plt.imshow(tau_arr_log[i_st+1:i_f, i_st+1:i_f], extent=[phi_0[i_st+1]/np.pi, phi_0[i_f]/np.pi,\
                                                    phi_0[i_st+1]/np.pi, phi_0[i_f]/np.pi],
           origin='lower', aspect='auto', cmap='viridis', vmin=1)
plt.title(r"$L = {}\Delta^{{-1}}, \tau = {}, \eta_d = {}, $"\
          .format(L, np.round(transp, 2), damping_param)+'\n'+
          r"$\Omega={}\Delta, T_{{qp}}^{{-1}} = {}\Delta, T_b^{{-1}} = {}\Delta$"\
          .format(Omega, beta_ferm, 1/T_bos), fontsize=13)

cbar = plt.colorbar()
cbar.set_label(r'$\log_{10}(\tau)$')  # Color bar label in log scale


plt.xlabel(r'$\varphi_{final}$')
plt.ylabel(r'$\varphi_{init}$')

plt.show()

In [ ]:
logtau_df = pd.DataFrame(
    tau_arr_log[i_st+2:i_f, i_st+2:i_f],
    columns=[f"log_tau_column_{i}" for i in range(tau_arr_log[i_st+2:i_f, i_st+2:i_f].shape[1])]
)

# Add a column for phi_0
logtau_df.insert(0, "phi_0", phi_0[i_st+2:i_f]/np.pi)

# Save to a text file
logtau_df.to_csv("log_tau_0.1.txt", sep="\t", index=False)

In [ ]:
tau_df = pd.DataFrame(
    tau_arr[i_st+1:i_f, i_st+1:i_f],
    columns=[f"tau_column_{i}" for i in range(tau_arr[i_st+1:i_f, i_st+1:i_f].shape[1])]
)

D0_df = pd.DataFrame(
    init_trace_dist[i_st+1:i_f, i_st+1:i_f],
    columns=[f"D0_column_{i}" for i in range(init_trace_dist[i_st+1:i_f, i_st+1:i_f].shape[1])]
)

# Add a column for phi_0
tau_df.insert(0, "phi_0", phi_0[i_st+1:i_f]/np.pi)

# Save to a text file
tau_df.to_csv("tau_0.1.txt", sep="\t", index=False)

In [ ]:
with open("tau_0.99.txt", "w") as file:
    # Write the header
    file.write("phi_init phi_f tau D0\n")
    
    # Iterate over the array indices and write the data
    for i in range(i_st+1, i_f):
        for j in range(i_st+1, i_f):
            file.write(f"{phi_0[i]} {phi_0[j]} {tau_arr[i, j]} {init_trace_dist[i, j]}\n")

### Time evolution with the eigenvalues of the steady state matrix:
compare the numerical time evolution to the time evolution given by the eigenvalues/vectors of the
steady state population matrix

In [ ]:
def eig_M(M):
    
    eigval_M = np.zeros((4), dtype='complex')
    eigvec_M = np.zeros((4, 4), dtype='complex')
    
    eigval_M, eigvec_M = LA.eig(M)
        
    return eigval_M, eigvec_M

In [ ]:
def find_c(init, eigvec_M):
    
    coeff = np.zeros((4))
    
    Matr = np.zeros((4, 4))
        
    for j in range(4):
        Matr[j] = eigvec_M[:, j]
            
    coeff = init @ LA.inv(Matr)
        
    return coeff

In [ ]:
def time_dep(t, init, eigval_M, eigvec_M):
    P_t = np.zeros((4))
    coeff = find_c(init, eigvec_M)
    
    for j in range(4):
        P_t[j] = np.sum(np.exp(eigval_M * t) * coeff @ eigvec_M[j, :])
        
        
    return P_t       

In [ ]:
eigval_M, eigvec_M = eig_M(Steady_M)
for j in range(4):
    plt.scatter(phi_0/np.pi, eigval_M[:, j], s=5)

In [ ]:
time_test = np.linspace(1, 200, num=100)
P_t = np.zeros((len(time_test), 4))
P_t1 = np.zeros((len(time_test), 4))
P_t2 = np.zeros_like(P_t)
P_t3 = np.zeros_like(P_t)
P_t4 = np.zeros_like(P_t)
P_t5 = np.zeros_like(P_t)
P_t6 = np.zeros_like(P_t)

In [ ]:
for t in range(len(time_test)):
    P_t1[t] = time_dep(time_test[t], steady_occup[80], eigval_M[65], eigvec_M[65])
    P_t2[t] = time_dep(time_test[t], steady_occup[88], eigval_M[65], eigvec_M[65]) 
    P_t3[t] = time_dep(time_test[t], steady_occup[90], eigval_M[65], eigvec_M[65]) 
    P_t[t] = time_dep(time_test[t], steady_occup[93], eigval_M[65], eigvec_M[65])
    P_t4[t] = time_dep(time_test[t], steady_occup[97], eigval_M[65], eigvec_M[65])
    P_t5[t] = time_dep(time_test[t], steady_occup[100], eigval_M[65], eigvec_M[65])   
    P_t6[t] = time_dep(time_test[t], steady_occup[94], eigval_M[65], eigvec_M[65])                    

In [ ]:
def compute_tau_test(i1, i2):
    
    P0 = steady_occup[i1]
    time_test = np.linspace(1, 2000, num=200)
    

    tau = 0  # Default value for tau
    for j in range(len(time_test)):
        ys_ = time_dep(time_test[j], P0, eigval_M[i2], eigvec_M[i2])
        if np.sum(np.abs(ys_[:] - steady_occup[i2, :])) < 1e-4:
            tau = time_test[j]
            break
            
    return i1, i2, tau

In [ ]:
tau_arr_test = np.zeros((len(phi_0), len(phi_0)))

In [ ]:
# Parallel execution
with ProcessPoolExecutor() as executor:
    # Submit tasks for all (i1, i2) pairs
    futures = [executor.submit(compute_tau_test, i1, i2) for i1 in range(mid+1) for i2 in range(mid+1)]
    
    # Collect results and populate tau_arr
    for future in futures:
        i1, i2, tau = future.result()
        tau_arr_test[i1, i2] = tau

# Now tau_arr contains the computed tau values

In [ ]:
N = len(phi_0)
for i in range(mid + 1):
    for j in range(mid + 1):
        value = tau_arr_test[i, j]
        
        # Reflect to the other quadrants
        tau_arr_test[i, N - 1 - j] = value          # Top-right
        tau_arr_test[N - 1 - i, j] = value          # Bottom-left
        tau_arr_test[N - 1 - i, N - 1 - j] = value  # Bottom-right

### Compute the exact time evolution of each point

In [ ]:
# Helper function to compute trace distance for each (i1, i2) pair

def compute_trace_distance(i1, i2, phi_0, steady_occup):
    
    time_points = np.linspace(0.0, 450.0, 500)
    
    P0 = steady_occup[i1]
    sol_ = solve_ivp(rhs1, (0.0, 450), P0, method='LSODA', t_eval=time_points, dense_output=True, args=(i2,))
    
    ts_ = sol_.t
    ys_ = np.stack(sol_.y)
    
    steady_reshaped = np.tile(steady_occup[i2, :, np.newaxis], (1, len(ts_)))
    trace_distance = np.sum(np.abs(ys_ - steady_reshaped), axis=0)
    
    return i1, i2, trace_distance, ts_

In [ ]:
# Initialize trace_dist with the correct shape (assuming it's a 2D array)

trace_dist = np.zeros((len(phi_0), len(phi_0)), dtype=object)
time = np.zeros((len(phi_0), len(phi_0)), dtype=object)

In [ ]:
# Use ProcessPoolExecutor to parallelize the computation

with ProcessPoolExecutor() as executor:
    futures = [
        executor.submit(compute_trace_distance, i1, i2, phi_0, steady_occup)
        for i1 in range(len(phi_0))
        for i2 in range(len(phi_0))
    ]
    for future in futures:
        i1, i2, trace_distance, ts_ = future.result()
        trace_dist[i1][i2] = trace_distance
        time[i1][i2] = ts_        

Plot the time evolution

In [ ]:
plt.plot(time[60][70], trace_dist[60][70], label=r"$\varphi_i = {}, \varphi_f = {}$".\
        format(np.round(phi_0[60]/np.pi, 2), np.round(phi_0[70]/np.pi, 2)))

plt.plot(time[50][70], trace_dist[50][70], label=r"$\varphi_i = {}, \varphi_f = {}$".\
        format(np.round(phi_0[50]/np.pi, 2), np.round(phi_0[70]/np.pi, 2)))

plt.plot(time[45][70], trace_dist[45][70], label=r"$\varphi_i = {}, \varphi_f = {}$".\
        format(np.round(phi_0[45]/np.pi, 2), np.round(phi_0[70]/np.pi, 2)))

plt.plot(time[41][70], trace_dist[41][70], label=r"$\varphi_i = {}, \varphi_f = {}$".\
        format(np.round(phi_0[41]/np.pi, 2), np.round(phi_0[70]/np.pi, 2)))

plt.plot(time[48][70], trace_dist[48][70], label=r"$\varphi_i = {}, \varphi_f = {}$".\
        format(np.round(phi_0[48]/np.pi, 2), np.round(phi_0[70]/np.pi, 2)))


plt.yscale('log')
plt.ylim(10**(-3), 1)
plt.xlim(-5, 450)


plt.title(r"$L = {}\Delta^{{-1}}, \tau = {}, \eta_d = {}, $"\
          .format(L, np.round(transp, 2), damping_param)+'\n'+
          r"$\Omega={}\Delta, T_{{qp}}^{{-1}} = {}\Delta, T_b^{{-1}} = {}\Delta$"\
          .format(Omega, beta_ferm, 1/T_bos), fontsize=13)


plt.legend()
plt.grid()

In [ ]:
plt.plot(time[86][80], trace_dist[86][80], label=r"$\varphi_i = {}, \varphi_f = {}$".\
        format(np.round(phi_0[86]/np.pi, 2), np.round(phi_0[80]/np.pi, 2)))

plt.plot(time[90][80], trace_dist[90][80], label=r"$\varphi_i = {}, \varphi_f = {}$".\
        format(np.round(phi_0[90]/np.pi, 2), np.round(phi_0[80]/np.pi, 2)))

plt.plot(time[93][80], trace_dist[93][80], label=r"$\varphi_i = {}, \varphi_f = {}$".\
        format(np.round(phi_0[93]/np.pi, 2), np.round(phi_0[80]/np.pi, 2)))

plt.plot(time[94][80], trace_dist[94][80], label=r"$\varphi_i = {}, \varphi_f = {}$".\
        format(np.round(phi_0[94]/np.pi, 2), np.round(phi_0[80]/np.pi, 2)))

plt.plot(time[95][80], trace_dist[95][80], label=r"$\varphi_i = {}, \varphi_f = {}$".\
        format(np.round(phi_0[95]/np.pi, 2), np.round(phi_0[80]/np.pi, 2)))



plt.yscale('log')
plt.ylim(10**(-3), 1)
plt.xlim(-5, 250)


plt.title(r"$L = {}\Delta^{{-1}}, \tau = {}, \eta_d = {}, $"\
          .format(L, np.round(transp, 2), damping_param)+'\n'+
          r"$\Omega={}\Delta, T_{{qp}}^{{-1}} = {}\Delta, T_b^{{-1}} = {}\Delta$"\
          .format(Omega, beta_ferm, 1/T_bos), fontsize=13)


plt.legend()
plt.grid()

In [ ]:
# Indices for the plots
indices = [86, 90, 93, 94, 95]
target = 80

# Prepare header for CSV
header = []
for idx in indices:
    label = f"phi_i={np.round(phi_0[idx]/np.pi, 2)}, phi_f={np.round(phi_0[target]/np.pi, 2)},"
    header.extend([f"{label} t", f"{label} D_T(t)"])

# Prepare rows (column-wise arrangement of data)
max_length = max(len(time[idx][target]) for idx in indices)
rows = []

for i in range(max_length):
    row = []
    for idx in indices:
        if i < len(time[idx][target]):
            row.extend([time[idx][target][i], trace_dist[idx][target][i]])
        else:
            row.extend([None, None])  # Fill with None for missing data
    rows.append(row)

# Save to CSV
output_file = "Dt_data_2.txt"
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(header)
    # Write the rows
    writer.writerows(rows)